In [10]:
import tensorflow as tf

x=tf.experimental.numpy.random.randn(10)
x=tf.random.normal([10])
type(x.reshape((5,2)))

AttributeError: ignored

In [16]:
tf.range(10), np.arange(10), K.arange(10)

(<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 <tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>)

In [26]:
from PIL import Image
from math import floor
import numpy as np
import time
from functools import partial
from random import random
import os
from keras.datasets import mnist

os.environ["CUDA_VISIBLE_DEVICES"]="1"

im_size = 28
latent_size = 64
emb_size = 32
num_labels = 10
BATCH_SIZE = 64
directory = "mnist_my"
suff = 'jpg'
cmode = 'RGB'
channels = 3
size_adjusted = False

k_images = 3

cha = 16

# def noise(n):
#     return np.random.normal(0.0, 1.0, size = [n, latent_size])

# generate points in latent space as input for the generator
def noise(n):
    # generate points in the latent space
    #x_input = tf.experimental.numpy.random.randn(latent_size * n)
    z_input = tf.random.normal([n, latent_size])
    # reshape into a batch of inputs for the network
    #z_input = x_input.reshape(n, latent_size)
    # generate labels
    return z_input


# load fashion mnist images
def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = mnist.load_data()
    # expand to 3d, e.g. add channels
    X = np.expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    #X = (X - 127.5) / 127.5
    X = X / 255.0
    return [X, trainy]
 
# # select real samples
def get_batch(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = np.random.randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    #y = ones((n_samples, 1))
    return [X, labels]#, y

def get_test_batch(num):
    
    (_, _), (testX, testy) = mnist.load_data()
    testX = np.expand_dims(testX, axis=-1)
    ix = np.random.randint(0, testX.shape[0], num)
    
    X, labels = testX[ix], testy[ix]
    
    return [np.array(X).astype('float32') / 255.0, labels]

############
## erased code space
############


# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 50, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r %s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total:
        print()
        print()

from keras.layers import Conv2D, Dense, AveragePooling2D, Activation, Cropping2D, Dropout, BatchNormalization
from keras.layers import Reshape, UpSampling2D, Flatten, Input, add, Lambda, concatenate, LeakyReLU, multiply
from keras.layers import GlobalAveragePooling2D, average, Embedding, ZeroPadding2D
from keras.models import model_from_json, Model
from keras.initializers import VarianceScaling
from tensorflow.keras.optimizers import Adam
import keras.backend as K
import tensorflow as tf

def gradient_penalty_loss(y_true, y_pred, averaged_samples, weight):
    
    def _compute_gradients(tensor, var_list):
        grads = tf.gradients(tensor, var_list)
        return [grad if grad is not None else tf.zeros_like(var) for var, grad in zip(var_list, grads)]
    
    gradients = K.gradients(y_pred, averaged_samples)[0]
    #gradients = tf.gradients(y_pred, averaged_samples)[0]
    #gradients = _compute_gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradient_penalty = K.sum(gradients_sqr,
                              axis=K.arange(1, len(gradients_sqr.shape)))

    # (weight / 2) * ||grad||^2
    # Penalize the gradient norm
    return K.mean(gradient_penalty) * (weight / 2)

def hinge_d(y_true, y_pred):
    return K.mean(K.relu(1.0 - (y_true * y_pred)))

def w_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def g_block(inp, fil, u = True):

    if u:
        out = UpSampling2D(interpolation = 'bilinear')(inp)
    else:
        out = Activation('linear')(inp)

    skip = Conv2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(out)

    out = Conv2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(out)
    out = LeakyReLU(0.2)(out)

    out = Conv2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(out)
    out = LeakyReLU(0.2)(out)

    out = Conv2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(out)

    out = add([out, skip])
    out = LeakyReLU(0.2)(out)

    return out

def d_block(inp, fil, p = True):

    skip = Conv2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(inp)

    out = Conv2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(inp)
    out = LeakyReLU(0.2)(out)

    out = Conv2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(out)
    out = LeakyReLU(0.2)(out)

    out = Conv2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(out)

    out = add([out, skip])
    out = LeakyReLU(0.2)(out)

    if p:
        out = AveragePooling2D()(out)

    return out

class GAN(object):

    def __init__(self, steps = 1, lr = 0.0001, decay = 0.00001):

        #Models
        self.D = None
        self.E = None
        self.G = None

        self.GE = None
        self.EE = None

        self.DM = None
        self.AM = None

        #Config
        self.LR = lr
        self.steps = steps
        self.beta = 0.999

        #Init Models
        self.discriminator()
        self.generator()
        self.encoder()

        self.EE = model_from_json(self.E.to_json())
        self.EE.set_weights(self.E.get_weights())

        self.GE = model_from_json(self.G.to_json())
        self.GE.set_weights(self.G.get_weights())

    def discriminator(self):

        if self.D:
            return self.D

        img_inp = Input(shape = [im_size, im_size, 1])
        z_inp = Input(shape = [latent_size])
        l_inp = Input(shape = (1,))
    
        #Label input
        n_nodes = 32 * 32
        li = Embedding(num_labels, emb_size)(l_inp)
        li = Dense(n_nodes)(li)
        li = Reshape((32, 32, 1))(li)

        #Latent input
        l = Dense(512, kernel_initializer = 'he_normal')(z_inp)
        l = LeakyReLU(0.2)(l)
        l = Dense(512, kernel_initializer = 'he_normal')(l)
        l = LeakyReLU(0.2)(l)
        l = Dense(512, kernel_initializer = 'he_normal')(l)
        l = LeakyReLU(0.2)(l)
        
        x = ZeroPadding2D((2, 2))(img_inp)
        x = concatenate([x, li])  # 32 32 2
        x = d_block(x, 2 * cha)  # 16 16 64
        x = d_block(x, 4 * cha)  # 8 8 128
        x = d_block(x, 8 * cha)  # 4 4 256
        x = d_block(x, 16 * cha, p = False)  # 4 4 512

        x = Flatten()(x)

        x = concatenate([x, l])

        x = Dense(16 * cha, kernel_initializer = 'he_normal')(x)
        x = LeakyReLU(0.2)(x)

        x = Dense(1, kernel_initializer = 'he_normal')(x)

        self.D = Model(inputs = [img_inp, z_inp, l_inp], outputs = x)

        return self.D

    def generator(self):

        if self.G:
            return self.G

        #Inputs
        z_inp = Input(shape = [latent_size])
        l_inp = Input(shape = (1,))

        #Label input
        n_nodes = 4 * 4
        li = Embedding(num_labels, emb_size)(l_inp)
        li = Dense(n_nodes)(li)
        li = Reshape((4, 4, 1))(li)

        #Actual Model
        x = Dense(4*4*16*cha, kernel_initializer = 'he_normal')(z_inp)
        x = Reshape([4, 4, 16*cha])(x)
        
        x = concatenate([x, li])  # 4 4 257

        x = g_block(x, 16 * cha, u = False)  # 4 4 256
        x = g_block(x, 8 * cha)  # 8 8 128
        x = g_block(x, 4 * cha)  # 16 16 64
        x = g_block(x, 3 * cha)  # 32 32 47
        x = Conv2D(filters = 1, kernel_size = 1, activation = 'sigmoid', padding = 'same', kernel_initializer = 'he_normal')(x)

        # cropping
        x = Cropping2D((2, 2))(x)
        
        self.G = Model(inputs = [z_inp, l_inp], outputs = x)

        return self.G

    def encoder(self):

        if self.E:
            return self.E

        img_inp = Input(shape = [im_size, im_size, 1])
        l_inp = Input(shape = (1,))

        #Label input
        n_nodes = 32 * 32
        li = Embedding(num_labels, emb_size)(l_inp)
        li = Dense(n_nodes)(li)
        li = Reshape((32, 32, 1))(li)

        x = ZeroPadding2D((2, 2))(img_inp)
        x = concatenate([x, li])  # 32 32 2
        x = d_block(x, 2 * cha)  # 16 16 64
        x = d_block(x, 4 * cha)  # 8 8 128
        x = d_block(x, 8 * cha)  # 4 4 256
        x = d_block(x, 16 * cha, p = False)  # 4 4 512

        x = Flatten()(x)

        x = Dense(16 * cha, kernel_initializer = 'he_normal')(x)
        x = LeakyReLU(0.2)(x)

        x = Dense(latent_size, kernel_initializer = 'he_normal', bias_initializer = 'zeros')(x)
        self.E = Model(inputs = [img_inp, l_inp], outputs = x)

        return self.E

    def AdModel(self):

        #D does not update
        self.D.trainable = False
        for layer in self.D.layers:
            layer.trainable = False

        #G does update
        self.G.trainable = True
        for layer in self.G.layers:
            layer.trainable = True

        #E does update
        self.E.trainable = True
        for layer in self.E.layers:
            layer.trainable = True

        # Fake Latent / Real Image / Real Label
        img_r = Input(shape = [im_size, im_size, 1])
        l_inp_r = Input(shape=(1,))
    
        enc_r = self.E([img_r, l_inp_r])
        logit_r = self.D([img_r, enc_r, l_inp_r])
        
        # Real Latent / Fake Image / Fake Label
        z = Input(shape = [latent_size])
        l_inp_f = Input(shape=(1,))

        img_f = self.G([z, l_inp_f])
        logit_f = self.D([img_f, z, l_inp_f])

        self.AM = Model(inputs = [img_r, z, l_inp_r, l_inp_f], outputs = [logit_r, logit_f])

        self.AM.compile(optimizer = Adam(self.LR, beta_1 = 0, beta_2 = 0.099), loss = [w_loss, w_loss], experimental_run_tf_function=False)

        return self.AM

    def DisModel(self):

        #D does update
        self.D.trainable = True
        for layer in self.D.layers:
            layer.trainable = True

        #G does not update
        self.G.trainable = False
        for layer in self.G.layers:
            layer.trainable = False

        #E does update
        self.E.trainable = False
        for layer in self.E.layers:
            layer.trainable = False

        # Fake Latent / Real Image / Real Label
        img_r = Input(shape = [im_size, im_size, 1])
        l_inp_r = Input(shape=(1,))
    
        enc_r = self.E([img_r, l_inp_r])
        logit_r = self.D([img_r, enc_r, l_inp_r])
        
        # Real Latent / Fake Image / Fake Label
        z = Input(shape = [latent_size])
        l_inp_f = Input(shape=(1,))

        img_f = self.G([z, l_inp_f])
        logit_f = self.D([img_f, z, l_inp_f])

        #self.DM = Model(inputs = [img_r, z, l_inp_r, l_inp_f], outputs = [logit_r, logit_f, logit_f])
        self.DM = Model(inputs = [img_r, z, l_inp_r, l_inp_f], outputs = [logit_r, logit_f])

        # Create partial of gradient penalty loss
        # For r1, averaged_samples = ri
        # For r2, averaged_samples = gf
        # Weight of 10 typically works
        partial_gp_loss = partial(gradient_penalty_loss, averaged_samples = [img_f, z, l_inp_f], weight = 10)
        partial_gp_loss.__name__ = 'partial_gp'  #https://github.com/elastic/apm-agent-python/issues/293
        
        #Compile With Corresponding Loss Functions
        #self.DM.compile(optimizer = Adam(self.LR, beta_1 = 0, beta_2 = 0.909), loss=[hinge_d, hinge_d, partial_gp_loss])
        self.DM.compile(optimizer = Adam(self.LR, beta_1 = 0, beta_2 = 0.909), loss=[hinge_d, hinge_d])

        return self.DM

    def EMA(self):

        start = time.clock()

        for i in range(len(self.G.layers)):
            up_weight = self.G.layers[i].get_weights()
            old_weight = self.GE.layers[i].get_weights()
            new_weight = []
            for j in range(len(up_weight)):
                new_weight.append(old_weight[j] * self.beta + (1-self.beta) * up_weight[j])
            self.GE.layers[i].set_weights(new_weight)

        for i in range(len(self.E.layers)):
            up_weight = self.E.layers[i].get_weights()
            old_weight = self.EE.layers[i].get_weights()
            new_weight = []
            for j in range(len(up_weight)):
                new_weight.append(old_weight[j] * self.beta + (1-self.beta) * up_weight[j])
            self.EE.layers[i].set_weights(new_weight)

        #print("Moved Average. " + str(time.clock() - start) + "s")

    def MAinit(self):
        self.EE.set_weights(self.E.get_weights())
        self.GE.set_weights(self.G.get_weights())


class BiGAN(object):

    def __init__(self, steps = 1, lr = 0.0001, decay = 0.00001, silent = True):

        self.GAN = GAN(steps = steps, lr = lr, decay = decay)
        self.DisModel = self.GAN.DisModel()
        self.AdModel = self.GAN.AdModel()

        self.lastblip = time.clock()

        self.noise_level = 0

        self.im = load_real_samples()

        self.silent = silent

        #Train Generator to be in the middle, not all the way at real. Apparently works better??
        self.ones = np.ones((BATCH_SIZE, 1), dtype=np.float32)
        self.zeros = np.zeros((BATCH_SIZE, 1), dtype=np.float32)
        self.nones = -self.ones

    def train(self):

        #Train Alternating
        a = self.train_dis()
        b = self.train_gen()

        if self.GAN.steps % 10 == 0:
            self.GAN.EMA()

        if self.GAN.steps == 20000:
            self.GAN.MAinit()


        #Print info
        if self.GAN.steps % 100 == 0 and not self.silent:
            print("\n\nRound " + str(self.GAN.steps) + ":")
            print("D: " + str(a))
            print("G: " + str(b))
            s = round((time.clock() - self.lastblip), 4)
            steps_per_second = 100 / s
            steps_per_minute = steps_per_second * 60
            steps_per_hour = steps_per_minute * 60
            print("Steps/Second: " + str(round(steps_per_second, 2)))
            print("Steps/Hour: " + str(round(steps_per_hour)))
            min1k = floor(1000/steps_per_minute)
            sec1k = floor(1000/steps_per_second) % 60
            print("1k Steps: " + str(min1k) + ":" + str(sec1k))
            self.lastblip = time.clock()
            steps_left = 200000 - self.GAN.steps + 1e-7
            hours_left = steps_left // steps_per_hour
            minutes_left = (steps_left // steps_per_minute) % 60

            print("Til Completion: " + str(int(hours_left)) + "h" + str(int(minutes_left)) + "m")
            print()

            #Save Model
            if self.GAN.steps % 500 == 0:
                self.save(floor(self.GAN.steps / 10000))
            if self.GAN.steps % 1000 == 0 or (self.GAN.steps % 100 == 0 and self.GAN.steps < 1000):
                self.evaluate(floor(self.GAN.steps / 1000))


        printProgressBar(self.GAN.steps % 100, 99, decimals = 0)

        self.GAN.steps = self.GAN.steps + 1

    def train_dis(self):
        
        train_X, train_y = get_batch(self.im, BATCH_SIZE)
        z_inp = noise(BATCH_SIZE)
        label_fake = tf.experimental.numpy.random.randint(0, num_labels, BATCH_SIZE)
        
        #Get Data
        # train_data = [self.im.get_batch(BATCH_SIZE), noise(BATCH_SIZE)]
        train_data = [train_X, z_inp, train_y, label_fake]

        #Train
        #d_loss = self.DisModel.train_on_batch(train_data, [self.ones, self.nones, self.ones])
        d_loss = self.DisModel.train_on_batch(train_data, [self.ones, self.nones])

        return d_loss

    def train_gen(self):

        train_X, train_y = get_batch(self.im, BATCH_SIZE)
        z_inp = noise(BATCH_SIZE)
        label_fake = tf.experimental.numpy.random.randint(0, num_labels, BATCH_SIZE)

        #Train
        train_data = [train_X, z_inp, train_y, label_fake]

        g_loss = self.AdModel.train_on_batch(train_data, [self.ones, self.nones])

        return g_loss

    def evaluate(self, num = 0):

        n1 = noise(32)
        fake_labels = np.random.randint(0, num_labels, 32)
        generated_images = self.GAN.G.predict([n1, fake_labels])

        real_images, real_labels = get_test_batch(16)
        latent_codes = self.GAN.E.predict([real_images, real_labels], batch_size = BATCH_SIZE)
        reconstructed_images = self.GAN.G.predict([latent_codes, real_labels], batch_size = BATCH_SIZE)

        print("E Mean: " + str(np.mean(latent_codes)))
        print("E Std: " + str(np.std(latent_codes)))
        print("E Std Featurewise: " + str(np.mean(np.std(latent_codes, axis = 0))))
        print()
###########
# tmp erased code space
###########
    def saveModel(self, model, name, num):
        json = model.to_json()
        with open("Models/"+name+".json", "w") as json_file:
            json_file.write(json)

        model.save_weights("Models/"+name+"_"+str(num)+".h5")

    def loadModel(self, name, num):

        file = open("Models/"+name+".json", 'r')
        json = file.read()
        file.close()

        mod = model_from_json(json)
        mod.load_weights("Models/"+name+"_"+str(num)+".h5")

        return mod

    def save(self, num): #Save JSON and Weights into /Models/
        self.saveModel(self.GAN.G, "gen", num)
        self.saveModel(self.GAN.D, "dis", num)
        self.saveModel(self.GAN.E, "enc", num)

        self.saveModel(self.GAN.GE, "genMA", num)
        self.saveModel(self.GAN.EE, "encMA", num)


    def load(self, num): #Load JSON and Weights from /Models/
        steps1 = self.GAN.steps

        #Load Models
        self.GAN.G = self.loadModel("gen", num)
        self.GAN.D = self.loadModel("dis", num)
        self.GAN.E = self.loadModel("enc", num)

        self.GAN.GE = self.loadModel("genMA", num)
        self.GAN.EE = self.loadModel("encMA", num)

        self.GAN.steps = steps1

        self.DisModel = self.GAN.DisModel()
        self.AdModel = self.GAN.AdModel()




# if __name__ == "__main__":
model = BiGAN(lr = 0.0001, silent = False)
model.evaluate(0)

while model.GAN.steps <= 1000:
    model.train()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:435: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


E Mean: 0.09336724
E Std: 0.77219963
E Std Featurewise: 0.2372183



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:402: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


  |██████████████████████████████████████████████████| 100% 



Round 100:
D: [0.2808283865451813, 0.2681701183319092, 0.01265828125178814]
G: [7.432832717895508, 5.520326137542725, 1.9125065803527832]
Steps/Second: 0.13
Steps/Hour: 470
1k Steps: 127:40
Til Completion: 425h21m



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:466: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:475: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


E Mean: -0.3513738
E Std: 2.26091
E Std Featurewise: 0.4488153

  |██████████████████████████████████████████████████| 100% 



Round 200:
D: [1.7820934057235718, 0.0, 1.7820934057235718]
G: [3.7569432258605957, 0.86772620677948, 2.889216899871826]
Steps/Second: 0.13
Steps/Hour: 475
1k Steps: 126:11
Til Completion: 420h12m

E Mean: 0.2846715
E Std: 1.3066609
E Std Featurewise: 0.28642145

  |██████████████████████████████████████████████████| 100% 



Round 300:
D: [1.2337967157363892, 0.018081584945321083, 1.2157151699066162]
G: [2.4081199169158936, -0.6714942455291748, 3.0796141624450684]
Steps/Second: 0.13
Steps/Hour: 473
1k Steps: 126:55
Til Completion: 422h26m

E Mean: -0.2665979
E Std: 1.6190413
E Std Featurewise: 0.42011935

  |██████████████████████████████████████████████████| 100% 



Round 400:
D: [1.1126668453216553, 0.1878311187028885, 0.9248356819152832]
G: [1.9985939264297485, 0.6559320688247681, 1.3426618576049805]
Steps/Second: 0.13
Steps/Hour: 469
1k Steps: 128:2
Til 

FileNotFoundError: ignored

In [6]:
import numpy as np

np.random.randint(0, 10, 10)

array([2, 4, 1, 1, 7, 9, 5, 7, 3, 0])